#### Comparison of average estimated score about good and bad combinations with size constraints

In [1]:
#%% GSE111818 optimization
import copy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('C:/github/LiverDeconv')
import liver_deconv as ld
from _utils import processing as pc
from _utils import plot4deconv

In [2]:
Base_dir = 'C:/github/LiverDeconv' # cloning repository
# load data
mix_df = pd.read_csv('C:/github/LiverDeconv/_Figures_new/Figure_5/data/gse111828_exp_matrix.csv',index_col=0)
ref = pd.read_csv(Base_dir + '/data/processed/ref_13types.csv',index_col=0)

comb_df = pd.read_pickle('C:/github/LiverDeconv/_Figures_new/Figure_4/data/mon_fluctuate_quartile_comb2ref.pkl')
comb_df = comb_df.sort_values("Monocyte",ascending=False)
comb = comb_df['pair'].tolist()

***
##### Good combinations

In [3]:
#%% good mean vs bad mean
n = 10
# good
good_summary = pd.DataFrame()
good_size = []
for i in range(n):
    use_cell = ['Monocyte']
    target_cells = copy.deepcopy(use_cell)
    #target_cells.extend([t.split("'")[0] for t in comb[i][2:-2].split("', '")])
    target_cells.extend(list(comb[i]))
    good_size.append(len(target_cells))
    # select use sample
    use_sample = []
    for t in ref.columns.tolist():
        if t.split("_")[0] in target_cells:
            use_sample.append(t)
    use_ref = ref[use_sample]
    
    # define reference
    dat = ld.LiverDeconv()
    dat.set_data(df_mix=mix_df,df_all=use_ref)
    dat.pre_processing(do_ann=False,ann_df=None,do_log2=True,do_quantile=True,do_trimming=False,do_drop=True)
    dat.narrow_intersec()
    dat.create_ref(sep="_",number=50,limit_CV=10,limit_FC=1.5,log2=False,verbose=False)
    
    df_mix = dat.df_mix
    df_all = dat.df_all
    
    final_ref = dat.final_ref
    
    dat.do_fit(file_dat=mix_df,file_ref=final_ref,max_iter=1e6,number_of_repeats=3,alpha=1,l1_ratio=0.05)
    res = dat.get_res()
    z_good = pc.standardz_sample(res)
    z_good.index = [i.split(" ")[0] for i in z_good.index]
    ctrl_mean = z_good.loc["0hr"]["Monocyte"].mean()
    fxn = lambda x : x-ctrl_mean
    z_good['Monocyte'] = z_good['Monocyte'].apply(fxn)
    good_summary[i] = z_good['Monocyte']

good_summary['name'] = good_summary.index.tolist()
good_melt = pd.melt(good_summary.groupby('name').mean().T)
good_melt['status']=['good']*len(good_melt)
good_melt.columns = ['variable','value','status']

good_min = min(good_size)

# save
good_melt.to_csv('C:/github/LiverDeconv/_Figures_new/Figure_5/results/mon_good_melt.csv')

(12076, 24)
(51463, 38)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 11930
signature genes : 200
0 rows are removed
0 rows are removed
number of used genes = 200
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 48)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12030
signature genes : 250
0 rows are removed
0 rows are removed
number of used genes = 250
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 43)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12026
signature genes : 200
0 rows are removed
0 rows are removed
number of used genes = 200
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 27)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 11856
signature genes : 150
0 rows are removed
0 rows are removed
number of used genes = 150
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 38)


***
##### Bad combinations

In [4]:
#%%
# bad
bad_summary = pd.DataFrame()
c = 0
bad_size = []
for i in range(100):
    if c == n:
        break
    use_cell = ['Monocyte']
    target_cells = copy.deepcopy(use_cell)
    #target_cells.extend([t.split("'")[0] for t in comb[-i][2:-2].split("', '")]) # -i th
    target_cells.extend(list(comb[-i]))
    if len(target_cells)<good_min:
        # skip if the target cell combination size is maller than good one
        continue
    bad_size.append(len(target_cells))
    # select use sample
    use_sample = []
    for t in ref.columns.tolist():
        if t.split("_")[0] in target_cells:
            use_sample.append(t)
    use_ref = ref[use_sample]
    
    # define reference
    dat = ld.LiverDeconv()
    dat.set_data(df_mix=mix_df,df_all=use_ref)
    dat.pre_processing(do_ann=False,ann_df=None,do_log2=True,do_quantile=True,do_trimming=False,do_drop=True)
    dat.narrow_intersec()
    dat.create_ref(sep="_",number=50,limit_CV=10,limit_FC=1.5,log2=False,verbose=False)
    
    df_mix = dat.df_mix
    df_all = dat.df_all
    
    final_ref = dat.final_ref
    
    dat.do_fit(file_dat=mix_df,file_ref=final_ref,max_iter=1e6,number_of_repeats=3,alpha=1,l1_ratio=0.05)
    res = dat.get_res()
    z_bad = pc.standardz_sample(res)
    z_bad.index = [i.split(" ")[0] for i in z_bad.index]
    ctrl_mean = z_bad.loc["0hr"]["Monocyte"].mean()
    fxn = lambda x : x-ctrl_mean
    z_bad['Monocyte'] = z_bad['Monocyte'].apply(fxn)
    bad_summary[i] = z_bad['Monocyte']
    c += 1
bad_summary['name'] = bad_summary.index.tolist()
bad_melt = pd.melt(bad_summary.groupby('name').mean().T)
bad_melt['status']=['bad']*len(bad_melt)
bad_melt.columns = ['variable','value','status']

# save
bad_melt.to_csv('C:/github/LiverDeconv/_Figures_new/Figure_5/results/mon_bad_melt.csv')

(12076, 24)
(51463, 38)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 11930
signature genes : 200
0 rows are removed
0 rows are removed
number of used genes = 200
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 42)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12039
signature genes : 200
0 rows are removed
0 rows are removed
number of used genes = 200
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 24)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12024
signature genes : 150
0 rows are removed
0 rows are removed
number of used genes = 150
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 41)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12037
signature genes : 250
0 rows are removed
0 rows are removed
number of used genes = 250
fitting method : ElasticNet
standardz population control
(12076, 24)
(51463, 48)


***
##### Baseline (13 cell types)

In [5]:
target_cells = ['B','Basophil','CD4','CD8','Cholangiocyte','Eosinophil','Hepatocyte','Kupffer','LSEC','Monocyte','NK','Neutrophil','Stellate']
# select use sample
use_sample = []
for t in ref.columns.tolist():
    if t.split("_")[0] in target_cells:
        use_sample.append(t)
use_ref = ref[use_sample]

# define reference
dat = ld.LiverDeconv()
dat.set_data(df_mix=mix_df,df_all=use_ref)
dat.pre_processing(do_ann=False,ann_df=None,do_log2=True,do_quantile=True,do_trimming=False,do_drop=True)
dat.narrow_intersec()
dat.create_ref(sep="_",number=50,limit_CV=10,limit_FC=1.5,log2=False,verbose=False)

df_mix = dat.df_mix
df_all = dat.df_all

final_ref = dat.final_ref

dat.do_fit(file_dat=mix_df,file_ref=final_ref,max_iter=1e6,number_of_repeats=3,alpha=1,l1_ratio=0.05)
res = dat.get_res()
z_total = pc.standardz_sample(res)
z_total.index = [i.split(" ")[0] for i in z_total.index]
ctrl_mean = z_total.loc["0hr"]["Monocyte"].mean()
fxn = lambda x : x-ctrl_mean
z_total['Monocyte'] = z_total['Monocyte'].apply(fxn)

total_summary = pd.DataFrame()
total_summary[0] = z_total['Monocyte']

total_summary['name'] = total_summary.index.tolist()
total_melt = pd.melt(total_summary.groupby('name').mean().T)
total_melt['status']=['total']*len(total_melt)
total_melt.columns = ['variable','value','status']

total_melt.to_csv('C:/github/LiverDeconv/_Figures_new/Figure_5/results/mon_baseline_melt.csv')


(12076, 24)
(51463, 113)
log2 conversion
quantile normalization
drop nan
narrowd gene number : 12039
signature genes : 521
0 rows are removed
0 rows are removed
number of used genes = 521
fitting method : ElasticNet
standardz population control
